In [49]:
import numpy as np
import pandas as pd
import os
import glob
import openpyxl
import xlrd

In [50]:
# 対象のディレクトリパスを指定
date_directory_path = r'C:/Users/hj7724/VScode/python/Jupyter_project/ga4entity_check/data/'

# ディレクトリ内のフォルダー名を取得
date_folder_list = [name for name in os.listdir(date_directory_path) if os.path.isdir(os.path.join(date_directory_path, name))]

if date_folder_list:
    grant_date = max(date_folder_list)
    print(f'Grant Date: {grant_date}')

Grant Date: 0716


In [51]:
#付与日フォルダのディレクトリ指定
path = "C:/Users/hj7724/VScode/python/Jupyter_project/ga4entity_check/data/{grant_date}".format(grant_date = grant_date)

#all_bookとしてmake_all_bookで作成したファイルを取得
all_book = pd.read_excel(path + '/all_book_{grant_date}.xlsx'.format(grant_date = grant_date))

In [52]:
# checkフォルダのパスを作成
check_path = os.path.join(path, "check")
# checkフォルダ内のExcelファイル名をリストで取得
csv_file_list = [os.path.basename(f) for f in glob.glob(os.path.join(check_path, "*.csv"))]

# 結果の表示
print(csv_file_list)

['entity_access_20250716_501.csv', 'entity_access_20250716_509.csv', 'entity_access_20250716_523.csv', 'entity_access_20250716_524.csv', 'entity_access_20250716_525.csv', 'entity_access_20250716_533.csv', 'entity_access_20250716_550.csv', 'entity_access_20250716_551.csv', 'entity_access_20250716_552.csv', 'entity_access_20250716_553.csv', 'entity_access_20250716_555.csv', 'entity_access_20250716_556.csv', 'entity_access_20250716_557.csv', 'entity_access_20250716_560.csv', 'entity_access_20250716_568.csv', 'entity_access_20250716_573.csv', 'entity_access_20250716_574.csv', 'entity_access_20250716_579.csv', 'entity_access_20250716_581.csv', 'entity_access_20250716_582.csv', 'entity_access_20250716_596.csv', 'entity_access_20250716_597.csv', 'entity_access_20250716_599.csv', 'entity_access_20250716_600.csv', 'entity_access_20250716_606.csv', 'entity_access_20250716_612.csv', 'entity_access_20250716_614.csv', 'entity_access_20250716_615.csv', 'entity_access_20250716_619.csv', 'entity_acces

In [53]:
#ディレクトリを移動
os.chdir(check_path)
#メールリストのcsvを辞書形式に作る関数
def make_entity_dict(file_names):
    # 結果を格納する辞書
    entity_dict = {}

    # 各ファイルに対して操作を行う
    for file_name in file_names:
        # CSVファイルを読み込む
        df = pd.read_csv(file_name)

        # メール列だけを残して他の列を削除する
        df = df[['メール']]

        # ファイル名からプロパティの数字を抽出
        propety_value = os.path.basename(file_name).split('_')[3].replace('.csv', '')

        # 新たに'プロパティ'列を追加して、抽出した数字を格納する
        df['プロパティ'] = propety_value

        # 新しいデータフレームの名前を作成
        new_df_name = f'entity_{propety_value}'

        # データフレームを辞書に格納
        entity_dict[new_df_name] = df
    
    return entity_dict

In [54]:
entity_dict = make_entity_dict(csv_file_list)

In [55]:
#申請書のメールアドレスが含まれるか確認する関数
def check_email(row):
    num_propety = str(row['num_propety'])  # num_propety を文字列として扱う
    mail = row['mail']

    # データフレームの名前を作成
    df_name = f'entity_{num_propety}'

    # 該当するデータフレームが存在するかチェック
    if df_name in entity_dict:
        entity_df = entity_dict[df_name]
        # メールアドレスが含まれているかチェック
        return mail in entity_df['メール'].values
    return False  # デフォルトはFalseとする


In [56]:
# all_bookの各行に対してcheck列を設定
all_book['check'] = all_book.apply(check_email, axis=1)

In [57]:
# all_bookをExcelファイルとして保存する
save_path = os.path.join(path, 'all_book_check_{grant_date}.xlsx'.format(grant_date = grant_date))
all_book.to_excel(save_path, index=False)

# 保存完了メッセージ
print(f"Excel file saved successfully at {save_path}")

Excel file saved successfully at C:/Users/hj7724/VScode/python/Jupyter_project/ga4entity_check/data/0716\all_book_check_0716.xlsx
